In [38]:
import pandas as pd
import numpy as np
from tableone import TableOne
from src.constants import RELEVANT_DESA_BAD
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Epitope Database

In [39]:
path_cohort = '~/UMCUtrecht/KaplanMeier/DESAsurvival_original.csv'
path_antibody = '~/UMCUtrecht/20211104_mismatch_ep_db.pickle'
df_abs = pd.read_pickle(path_antibody)
# There is also another DESA column in the df_cohort.
df_cohort = pd.read_csv(path_cohort, sep=';').drop('DESA', axis=1)
df = df_abs.merge(df_cohort, on='TransplantID')

float_columns =  ['DialysisYears', 'CIPHour_DBD', 'CIPHour_DCD', 'CIPHour']
for col in float_columns:
    df[col] = df[col].apply(lambda x: x.replace(',', '.')).astype('float')

In [93]:
# print(df.columns)
# print(df.shape)
for item in df.columns:
    if 'hemo' in item.lower():
        print(item) 
# df.columns[20:60]

###  Table 1

In [90]:
# selected variables
cols = [
    'DESA_Status', 
    'RecipientAge_NOTR', 'RecipientSex_NOTR', 'DialysisYears',
    'DonorAge_NOTR', 'DonorSex_NOTR', 
    # 'TypeOfDonor_NOTR',
    'TypeCadaveric_NOTR', 
    'CIPHour_DBD', 'CIPHour_DCD',
    'CIPHour',
    'Retransplant', 
    'CurrentPRA_NOTR',
    'HighPRA_NOTR',
    'ALG_ATG_OKT3_AntiPanTMoAb',
    'IL2rMoAb_T0',
    'NumMismatch_ABDR_broad',
    'Steroids_T0', 'MMF_T0', 'Aza_T0', 'Sirolimus_T0', 'Cyclosporin_T0', 'Tacrolimus_T0', 'Unknown_T0', 'Others_T0',
    
]
df_t1 = df[cols]
df_t1['DESA_Status'] = df_t1['DESA_Status'].apply(lambda x:'DESA' if x == 'DESA' else 'No DESA')
df_t1['TypeCadaveric_NOTR'].fillna('Living', inplace=True)

df_t1['CIPDeceadDonor'] = (df_t1['CIPHour_DBD'] + df_t1['CIPHour_DCD']).replace(0, np.nan)
for g, v in df.groupby('TypeOfDonor_NOTR')['CIPHour']:
    if g == 'Living':
        df_t1['CIPLivingDonor'] = v
df_t1.drop(['CIPHour_DBD', 'CIPHour_DCD', 'CIPHour'], axis =1, inplace=True)
df_t1['MMF/azathioprine'] = df_t1['MMF_T0'].eq('Yes') + df_t1['Aza_T0'].eq('Yes')
df_t1['Cyclosporin/Tacrolimus'] = df_t1['Cyclosporin_T0'].eq('Yes') + df_t1['Tacrolimus_T0'].eq('Yes') 
df_t1.drop(['Cyclosporin_T0', 'Tacrolimus_T0', 'MMF_T0', 'Aza_T0'], axis =1, inplace=True)

/var/folders/dz/h73npvns6dscvynm6d4050wc0000gn/T/ipykernel_40979/4152126546.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['DESA_Status'] = df_t1['DESA_Status'].apply(lambda x:'DESA' if x == 'DESA' else 'No DESA')
/var/folders/dz/h73npvns6dscvynm6d4050wc0000gn/T/ipykernel_40979/4152126546.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['TypeCadaveric_NOTR'].fillna('Living', inplace=True)
/var/folders/dz/h73npvns6dscvynm6d4050wc0000gn/T/ipykernel_40979/4152126546.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [92]:
# columns containing categorical variables
categorical = ['RecipientSex_NOTR', 'DonorSex_NOTR', 
'IL2rMoAb_T0', 'ALG_ATG_OKT3_AntiPanTMoAb',
'Retransplant', 'TypeCadaveric_NOTR',
'Steroids_T0', 'Sirolimus_T0', 'MMF/azathioprine', 'Cyclosporin/Tacrolimus', 'Unknown_T0', 'Others_T0',
]

# optionally, a categorical variable for stratification
groupby = ['DESA_Status']

# rename the death column
labels={
    'DonorAge_NOTR': 'Donor Age',
    'DonorSex_NOTR': 'Donor Sex',
    'RecipientAge_NOTR': 'Recipiet Age',
    'RecipientSex_NOTR': 'Recipiet Sex',
    'DialysisYears': 'Time on dialysis, y',
    'TypeCadaveric_NOTR': 'Type of Donor',
    'CIPDeceadDonor': ' Cold Ischemia Time Deceased Donors', 
    'CIPLivingDonor': ' Cold Ischemia Time Living Donors', 
    'CurrentPRA_NOTR': 'PRA at time of transplant, %',
    'HighPRA_NOTR': 'Highest PRA, %',
    'NumMismatch_ABDR_broad': 'HLA-A/B/DR broad mismatches, mean ± SD', 
}

continuous = {
    'DonorAge_NOTR', 'RecipientAge_NOTR', 'CIPDeceadDonor', 'DialysisYears',
    'CIPLivingDonor', 'CurrentPRA_NOTR', 'HighPRA_NOTR', 'NumMismatch_ABDR_broad',
    }
htest = {var:scipy.stats.mannwhitneyu for var in continuous}

# link to documentation https://github.com/tompollard/tableone/blob/master/tableone/tableone.py
import scipy
table1 = TableOne(
    df_t1, 
    groupby=groupby,
    missing=False,
    categorical=categorical,
    pval=True,
    rename=labels,
    htest_name=True,
    htest=htest,
)
table1

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use grou

Grouped by DESA_Status                                                  
                                                                                  Overall         DESA      No DESA P-Value            Test
n                                                                                    4690          439         4251                        
Recipiet Age, mean (SD)                                                       45.4 (14.4)  44.6 (13.7)  45.5 (14.4)   0.126    mannwhitneyu
Recipiet Sex, n (%)                               Female                      1879 (40.1)   267 (60.8)  1612 (37.9)  <0.001     Chi-squared
                                                  Male                        2811 (59.9)   172 (39.2)  2639 (62.1)                        
Time on dialysis, y, mean (SD)                                                  2.8 (2.5)    3.3 (3.0)    2.7 (2.4)   0.001    mannwhitneyu
Donor Age, mean (SD)                                                          44.3 (15.0)  45.1 (15.3)  44.2 (15.0)   0.087    mannwhitneyu
Donor Sex, n (%)                                  Female                      2366 (50.4)   193 (44.0)  2173 (51.1)   0.005     Chi-squared
                                                  Male                        2324 (49.6)   246 (56.0)  2078 (48.9)                        
Type of Donor, n (%)                              Heartbeating                2427 (51.7)   248 (56.5)  2179 (51.3)   0.111     Chi-squared
                                                  Living                      1455 (31.0)   124 (28.2)  1331 (31.3)                        
                                                  Non-heartbeating             808 (17.2)    67 (15.3)   741 (17.4)                        
Retransplant, n (%)                               No                          3970 (84.6)   239 (54.4)  3731 (87.8)  <0.001     Chi-squared
                                                  Yes                          720 (15.4)   200 (45.6)   520 (12.2)                        
PRA at time of transplant, %, mean (SD)                                        6.0 (17.5)  25.6 (31.7)   4.0 (13.8)  <0.001    mannwhitneyu
Highest PRA, %, mean (SD)                                                     13.9 (25.9)  43.8 (36.8)  10.8 (22.3)  <0.001    mannwhitneyu
ALG_ATG_OKT3_AntiPanTMoAb, n (%)                  No                          4506 (96.1)   409 (93.2)  4097 (96.4)   0.002     Chi-squared
                                                  Yes                           184 (3.9)     30 (6.8)    154 (3.6)                        
IL2rMoAb_T0, n (%)                                No                          3675 (78.4)   353 (80.4)  3322 (78.1)   0.300     Chi-squared
                                                  Yes                         1015 (21.6)    86 (19.6)   929 (21.9)                        
HLA-A/B/DR broad mismatches, mean ± SD, mean (SD)                               2.3 (1.5)    2.3 (1.4)    2.3 (1.5)   0.287    mannwhitneyu
Steroids_T0, n (%)                                No                            105 (2.2)     16 (3.6)     89 (2.1)   0.055     Chi-squared
                                                  Yes                         4585 (97.8)   423 (96.4)  4162 (97.9)                        
Sirolimus_T0, n (%)                               No                          4406 (93.9)   416 (94.8)  3990 (93.9)   0.517     Chi-squared
                                                  Yes                           284 (6.1)     23 (5.2)    261 (6.1)                        
Unknown_T0, n (%)                                 No                          4674 (99.7)   435 (99.1)  4239 (99.7)   0.056  Fisher's exact
                                                  Yes                            16 (0.3)      4 (0.9)     12 (0.3)                        
Others_T0, n (%)                                  No                          4090 (87.2)   404 (92.0)  3686 (86.7)   0.002     Chi-squared
       